# Clone the repo
Get the repo and submodules

In [1]:
!git clone https://github.com/nuwandda/yolov7-logo.git

Cloning into 'yolov7-logo'...
remote: Enumerating objects: 188, done.
remote: Counting objects: 100% (117/117), done.
remote: Compressing objects: 100% (90/90), done.
remote: Total 188 (delta 57), reused 73 (delta 25), pack-reused 71 (from 1)
Receiving objects: 100% (188/188), 78.76 MiB | 12.61 MiB/s, done.
Resolving deltas: 100% (73/73), done.


Download submodules.

In [2]:
%cd yolov7-logo/
!git submodule update --init

/content/yolov7-logo
Submodule 'src/yolov7' (https://github.com/WongKinYiu/yolov7.git) registered for path 'src/yolov7'
Cloning into '/content/yolov7-logo/src/yolov7'...
Submodule path 'src/yolov7': checked out '8c0bf3f78947a2e81a1d552903b4934777acfa5f'


Install the necessary packages.

In [3]:
!pip install -r src/requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 407.8/407.8 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 35.1 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.5
    Uninstalling protobuf-4.25.5:
      Successfully uninstalled protobuf-4.25.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-ai-generativelanguage 0.6.10 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0dev,>=3.20.2, but you have protobuf 4.21.2 which is incompatible.
google-api-core 2.19.2 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0.dev0,>=3.19.5, but you have protobuf 4.21.2 which is incompatible.
google-cloud-aiplatform 1.71.1 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0dev,>=3.20.2, but you have prot

# Download the dataset
Run the **getFlickr.sh** file to download the dataset.

In [10]:
!sh data/getFlickr.sh

--2024-12-06 01:16:20--  http://image.ntua.gr/iva/datasets/flickr_logos/flickr_logos_27_dataset.tar.gz
Resolving image.ntua.gr (image.ntua.gr)... 147.102.11.1
Connecting to image.ntua.gr (image.ntua.gr)|147.102.11.1|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 101827904 (97M) [application/x-gzip]
Saving to: ‘data/flickr_logos_27_dataset.tar.gz’

flickr_logos_27_dat 100%[===================>]  97.11M  14.7MB/s    in 17s     

2024-12-06 01:16:38 (5.61 MB/s) - ‘data/flickr_logos_27_dataset.tar.gz’ saved [101827904/101827904]

flickr_logos_27_dataset/
flickr_logos_27_dataset/flickr_logos_27_dataset_distractor_set_urls.txt
flickr_logos_27_dataset/flickr_logos_27_dataset_training_set_annotation.txt
flickr_logos_27_dataset/flickr_logos_27_dataset_query_set_annotation.txt
flickr_logos_27_dataset/flickr_logos_27_dataset_images.tar.gz
flickr_logos_27_dataset_images/
flickr_logos_27_dataset_images/2176584286.jpg
flickr_logos_27_dataset_images/3156402215.jpg
flickr_log

# Prepare data
We nee to convert data to YOLO format.Now that we have our dataset, we need to convert the annotations into the format expected by YOLOv7. YOLOv7 expects data to be organized in a specific way, otherwise it is unable to parse through the directories.

In [18]:
!python src/convert_annotations.py --dataset flickr27

100% 1079/1079 [00:04<00:00, 259.43it/s]


To see if the conversion is correct, run.

In [19]:
!python src/convert_annotations.py --dataset flickr27 --plot

100% 1079/1079 [00:04<00:00, 260.46it/s]
Figure(640x480)


Then, split data into sets.Next, we need to partition the dataset into train, validation, and test sets. These will contain 80%, 10%, and 10% of the data, respectively.

In [20]:
!python src/prepare_data.py --dataset flickr27

# Start training

If you want to use the GPU, there is some changes code before start this code, otherwise it will be error when executed.

Add on ```yolov7/utils/loss.py```

change @ line 685
```
from_which_layer.append((torch.ones(size=(len(b),)) * i).to('cuda'))
```

add code @ line 757
```
fg_mask_inboxes = fg_mask_inboxes.to(torch.device('cuda'))
```

In [21]:
!python src/yolov7/train.py --img-size 640 --cfg src/cfg/training/yolov7.yaml --hyp data/hyp.scratch.yaml --batch 2 --epoch 300 --data data/logo_data_flickr.yaml --weights src/yolov7_training.pt --workers 2 --name yolo_logo_det --device 0

2024-12-06 01:24:55.598851: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-06 01:24:55.636791: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-06 01:24:55.648176: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-06 01:24:55.678668: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-06 01:24:57.168437: W tensorflow/compiler/tf2